<a href="https://colab.research.google.com/github/AnandVP123/q1/blob/main/Final_Assignment_3_vanilla_model_tamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##installing wandb
!pip install wandb
##dowloading dakshin data set
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
##dakshin dataset
!tar -xf dakshina_dataset_v1.0.tar



     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 144 kB 53.4 MB/s 
     |████████████████████████████████| 181 kB 51.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ea126a966627aecee3b0de715072b5ef8a9b6073b8181f14cbee6072fbdfb5f6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
--2022-05-08 15:47:56--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.187.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[============

In [36]:

##########processingdakshin
import pandas as pd
import os
import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt
##Defineclass
class DP():
    def __init__(self, DATAPATH, source_lang = 'en', target_lang = "ta"):
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.trainpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.train.tsv")
        self.valpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.dev.tsv")
        self.testpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.test.tsv")
        self.train = pd.read_csv(self.trainpath,sep="\t",names=["tgt", "src", "count"],)
        self.val = pd.read_csv(self.valpath,sep="\t",names=["tgt", "src", "count"],)
        self.test = pd.read_csv(self.testpath,sep="\t",names=["tgt", "src", "count"],)
        ##Traijning of Data & creating val/test data 
        self.traindata = self.preprocess(self.train["src"].to_list(), self.train["tgt"].to_list())
        (self.trencoderinput,self.train_decoder_input,self.train_decoder_target,self.source_vocab,self.target_vocab,) = self.traindata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
        self.valdata = self.encode(self.val["src"].to_list(),self.val["tgt"].to_list(),list(self.so_chartoint.keys()),list(self.tar_chartoint.keys()),so_chartoint=self.so_chartoint,tar_chartoint=self.tar_chartoint,)
        self.valencoderinput, self.val_decoder_input, self.val_decoder_target = self.valdata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
        self.testdata = self.encode(self.test["src"].to_list(),self.test["tgt"].to_list(),list(self.so_chartoint.keys()),list(self.tar_chartoint.keys()),so_chartoint=self.so_chartoint,tar_chartoint=self.tar_chartoint,)
        self.testencoderinput, self.test_decoder_input, self.test_decoder_target = self.testdata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
    def dictionary_lookup(self, vocab):
        char2int = dict([(char, i) for i, char in enumerate(vocab)])
        int2char = dict((i, char) for char, i in char2int.items())
        return char2int, int2char
    def encode(self, source, target, source_chars, target_chars, so_chartoint=None, tar_chartoint=None):
        num_encoder_tokens = len(source_chars)
        numdecodertok = len(target_chars)
        sourcelengthmax = max([len(txt) for txt in source])
        targetlength = max([len(txt) for txt in target])
        source_vocab, target_vocab = None, None
        if so_chartoint == None and tar_chartoint == None:
            print("Generating the dictionary lookups for character to integer mapping and back")
            so_chartoint, source_int2char = self.dictionary_lookup(source_chars)
            tar_chartoint, target_int2char = self.dictionary_lookup(target_chars)
            source_vocab = (so_chartoint, source_int2char)
            target_vocab = (tar_chartoint, target_int2char)
        encoderindata = np.zeros((len(source), sourcelengthmax, num_encoder_tokens), dtype="float32")
        decoderindata = np.zeros((len(source), targetlength, numdecodertok), dtype="float32")
        decodertardata = np.zeros((len(source), targetlength, numdecodertok), dtype="float32")
        for i, (input_text, target_text) in enumerate(zip(source, target)):
            for t, char in enumerate(input_text):
                encoderindata[i, t, so_chartoint[char]] = 1.0
            encoderindata[i, t + 1 :, so_chartoint[" "]] = 1.0
            for t, char in enumerate(target_text):
                decoderindata[i, t, tar_chartoint[char]] = 1.0
                if t > 0:
                    decodertardata[i, t - 1, tar_chartoint[char]] = 1.0
            decoderindata[i, t + 1 :, tar_chartoint[" "]] = 1.0
            decodertardata[i, t:, tar_chartoint[" "]] = 1.0
        if source_vocab != None and target_vocab != None:return (encoderindata,decoderindata,decodertardata,source_vocab,target_vocab,)
        else:return encoderindata, decoderindata, decodertardata
    def preprocess(self, source , target):
        source_chars = set()
        target_chars = set()
        source = [str(x) for x in source]
        target = [str(x) for x in target]
        source_words = []
        target_words = []
        for src, tgt in zip(source, target):
            tgt = "\t" + tgt + "\n"
            source_words.append(src)
            target_words.append(tgt)
            for char in src:
                if char not in source_chars:
                    source_chars.add(char)
            for char in tgt:
                if char not in target_chars:
                    target_chars.add(char)
        source_chars = sorted(list(source_chars))
        target_chars = sorted(list(target_chars))
        source_chars.append(" ")
        target_chars.append(" ")
        num_encoder_tokens = len(source_chars)
        print("Source Vocab length:", num_encoder_tokens)
        numdecodertok = len(target_chars)
        print("Target Vocab length:", numdecodertok)
        sourcelengthmax = max([len(txt) for txt in source_words])
        print("Max sequence length for inputs:", sourcelengthmax)
        targetlength = max([len(txt) for txt in target_words])
        print("Max sequence length for outputs:", targetlength)
        print("Number of samples:", len(source))
        return self.encode(source_words, target_words, source_chars, target_chars)


In [37]:
#####processesing db for tamil

import os
import numpy as np
import pandas as pd
DATAPATH = "./dakshina_dataset_v1.0"
dataBase = DP(DATAPATH) 



Source Vocab length: 27
Target Vocab length: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28
Number of samples: 68218
Generating the dictionary lookups for character to integer mapping and back


In [30]:
####RNN

import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wandb
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model, Sequential,  Model
from tensorflow.keras.callbacks import EarlyStopping
class Sequence2Sequence():
    def __init__(self, modelConfigDict, srcChar2Int, tgtChar2Int, using_pretrained_model = False):
        #self.native_vocabulary = modelConfigDict["native_vocabulary"]
        self.nE = modelConfigDict["nE"]
        self.ct = modelConfigDict["ct"]
        self.ld = modelConfigDict["ld"]
        self.dout = modelConfigDict["dout"]
        self.nD = modelConfigDict["nD"]
        self.hidden = modelConfigDict["hidden"]
        self.tgtChar2Int = tgtChar2Int
        self.srcChar2Int = srcChar2Int
    ## encoder & decoder
    def build_configurable_model(self):       
    if self.ct == "RNN":
    einput = Input(shape=(None, len(self.srcChar2Int)))
    eout = einput
    for i in range(1, self.nE + 1):
    encoder = SimpleRNN(self.ld,retstate=True,retsequence=True,dout=self.dout,)
    eout, state = encoder(einput)estate = [state]
    dinput = Input(shape=(None, len(self.tgtChar2Int)))
   doutput = dinput
    for i in range(1, self.nD + 1):decoder = SimpleRNN(self.ld, retsequence=True, retstate=True, dout=self.dout,)
   doutput, _ = decoder(dinput, initial_state=estate)
  hidden = Dense(self.hidden, activation="relu")
  hidoutput = hidden(decoder_outputs)
  ddense = Dense(len(self.tgtChar2Int), activation="softmax")
  doutput = ddense(hidoutput)
  model = Model([einput, dinput],doutput)
 return model
elif self.ct == "LSTM":
einput = Input(shape=(None, len(self.srcChar2Int)))
eout = einput
for i in range(1, self.nE + 1):
encoder = LSTM(self.ld,retstate=True,retsequence=True,dout=self.dout,)
eout, state_h, state_c = encoder(eout)
estate = [state_h, state_c]
dinput = Input(shape=(None, len(self.tgtChar2Int)))
doutput = dinput
for i in range(1, self.nD + 1):
decoder = LSTM(self.ld,retstate=True,retsequence=True,dout=self.dout,)
doutput, _, _ = decoder(doutput, initial_state=estate)
### dense
hidden = Dense(self.hidden, activation="relu")
hidoutput = hidden(decoder_outputs)
ddense = Dense(len(self.tgtChar2Int), activation="softmax")
doutput = ddense(hidoutput)
model = Model([einput, dinput],doutput)
return model
elif self.ct == "GRU":
einput = Input(shape=(None, len(self.srcChar2Int)))
eout = einput
for i in range(1, self.nE + 1):
encoder = GRU(self.ld,retstate=True,retsequence=True,dout=self.dout,)
eout, state = encoder(einput)
estate = [state]
dinput = Input(shape=(None, len(self.tgtChar2Int)))
doutput = dinput
for i in range(1, self.nD + 1):
decoder = GRU(self.ld,retsequence=True,retstate=True,dout=self.dout,)
doutput, _ = decoder(dinput, initial_state=estate)
hidden = Dense(self.hidden, activation="relu")
hidoutput = hidden(decoder_outputs)
ddense = Dense(len(self.tgtChar2Int), activation="softmax")
doutput = ddense(hidoutput)
model = Model([einput, dinput],doutput)
return model

IndentationError: ignored

In [31]:
##model training
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import RNN, LSTM, GRU, Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import wandb
from wandb.keras import WandbCallback
physical_devices = tf.config.list_physical_devices('GPU')
try:tf.config.experimental.set_memory_growth(physical_devices[0], True)
except: pass
def train():
config_defaults = {"ct": "RNN","ld": 256,"hidden": 128,"optimiser": "rmsprop","nE": 1,"nD": 1,"dout": 0.2,"epochs": 1,"batchsize": 64,}
##wandbinit
wandb.init(config=config_defaults,  project="cs6910_a3", entity="ee20d064oe21d019")
config = wandb.config
wandb.run.name = (str(config.ct)
        + dataBase.source_lang
        + str(config.nE)
        + "_"
        + dataBase.target_lang
        + "_"
        + str(config.nD)
        + "_"
        + config.optimiser
        + "_"
        + str(config.epochs)
        + "_"
        + str(config.dout) 
        + "_"
        + str(config.batchsize)
        + "_"
        + str(config.ld))
    wandb.run.save()
modelInit = Sequence2Sequence(config,srcChar2Int=dataBase.so_chartoint, tgtChar2Int=dataBase.tar_chartoint)
model = modelInit.build_configurable_model()
model.summary()
model.compile(optimizer=config.optimiser,loss="categorical_crossentropy",metrics=["accuracy"],)
earlystopping = EarlyStopping(
monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto")
model.fit([dataBase.trencoderinput, dataBase.train_decoder_input],dataBase.train_decoder_target,batchsize=config.batchsize,epochs=config.epochs,validation_data=([dataBase.valencoderinput, dataBase.val_decoder_input], dataBase.val_decoder_target),callbacks=[earlystopping, WandbCallback()],)
model.save(os.DATAPATH.join("./TrainedModels", wandb.run.name))    
wandb.finish()
return model






IndentationError: ignored

In [ ]:
'''    
sweep_config = {
    "name": "Bayesian Sweep without attention",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
         "ct": {"values": ["RNN", "GRU", "LSTM"]},
         "ld": {"values": [256]},
         "hidden": {"values": [128, 64]},
         "optimiser": {"values": ["rmsprop", "adam"]},
         "nE": {"values": [1, 2, 3]},
         "nD": {"values": [1, 2, 3]},
         "dout": {"values": [0.1, 0.2, 0.3]},
         "epochs": {"values": [5,10,15]},
         "batchsize": {"values": [32, 64]},},}

sweep_id = wandb.sweep(sweep_config, project="cs6910_a3", entity="ee20d064oe21d019")
wandb.agent(sweep_id, train)

'''


'    \nsweep_config = {\n    "name": "Bayesian Sweep without attention",\n    "method": "bayes",\n    "metric": {"name": "val_accuracy", "goal": "maximize"},\n    "parameters": {\n        \n        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},\n        \n        "latentDim": {"values": [256]},\n        \n        "hidden": {"values": [128, 64]},\n        \n        "optimiser": {"values": ["rmsprop", "adam"]},\n        \n        "numEncoders": {"values": [1, 2, 3]},\n        \n        "numDecoders": {"values": [1, 2, 3]},\n        \n        "dropout": {"values": [0.1, 0.2, 0.3]},\n        \n        "epochs": {"values": [5,10,15]},\n        \n        "batch_size": {"values": [32, 64]},\n    },\n}\n\nsweep_id = wandb.sweep(sweep_config, project="cs6910_a3", entity="ee20d064oe21d019")\n\nwandb.agent(sweep_id, train)\n\n'

In [ ]:
  
sweep_config = {
    "name": "Bayesian Sweep without attention",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
                "ct": {"values": ["RNN", "GRU", "LSTM"]},
                "ld": {"values": [256]},
                "hidden": {"values": [128, 64]},
                "optimiser": {"values": ["rmsprop", "adam"]},
                "nE": {"values": [1, 2, 3]},
                "nD": {"values": [1, 2, 3]},
                "dout": {"values": [0.1, 0.2, 0.3]},
                "epochs": {"values": [5,10,15, 20]},
                "batchsize": {"values": [32, 64]},
    },}
sweep_id = wandb.sweep(sweep_config, project="cs6910_a3", entity="ee20d064oe21d019")
wandb.agent(sweep_id, train, count = 200)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: fzu4vby8
Sweep URL: https://wandb.ai/ee20d064oe21d019/cs6910_a3/sweeps/fzu4vby8


wandb: Agent Starting Run: kwe31qoz with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.2
wandb: 	epochs: 20
wandb: 	hidden: 64
wandb: 	latentDim: 256
wandb: 	numDecoders: 2
wandb: 	numEncoders: 3
wandb: 	optimiser: rmsprop
wandb: Currently logged in as: avp (ee20d064oe21d019). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

INFO:tensorflow:Assets written to: ./TrainedModels/GRUen3_ta_1_adam_20_0.2_64_256/assets


INFO:tensorflow:Assets written to: ./TrainedModels/GRUen3_ta_1_adam_20_0.2_64_256/assets


accuracy,▁▃▅▆▇▇▇▇█████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▆▄▃▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▅▇▇▇█▇████
val_loss,█▇▃▅▂▃▃▁▃▂▂▄▃
accuracy,0.94481
best_epoch,7
best_val_loss,1.57688
epoch,12
loss,0.18522
val_accuracy,0.784


wandb: Agent Starting Run: k6h5z4ng with config:
wandb: 	batch_size: 64
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.1
wandb: 	epochs: 15
wandb: 	hidden: 64
wandb: 	latentDim: 256
wandb: 	numDecoders: 3
wandb: 	numEncoders: 2
wandb: 	optimiser: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, None, 256),  290816      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 input_2 (InputLayer)           [(None, None, 49)]   0           []                               
                                                                                              

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp -rf ./TrainedModels /content/gdrive/MyDrive/CS6910/Assignment3/

Mounted at /content/gdrive
